# **Процедурная память с Mem0**

### **Подготовка и установка**

Для работы понадобится API ключ от сервиса Mem0 (его можно получить на [дашборде](https://app.mem0.ai/dashboard?spm=a2ty_o01.29997173.0.0.77705171Qo3xyp) и установленный Python SDK.

In [ ]:
!pip install mem0ai --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 276.0/276.0 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 197.8/197.8 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 390.4/390.4 kB 26.5 MB/s eta 0:00:00


### **Инициализация клиента**

Подключение к сервису происходит в одну строку кода с использованием ключа.


In [2]:
from mem0 import MemoryClient
import dotenv

API_KEY = dotenv.get_key("../.env", "MEM0_API_KEY")
client = MemoryClient(api_key=API_KEY)

### **Добавление памяти (Add Memory)**

Mem0 автоматически извлекает важные факты из сообщений. Не нужно вручную структурировать данные — просто необходимо передать диалог.

**Пример:**
Пользователь сообщает о проблеме с заказом.

In [3]:
user_id = "customer-001"
message_from_user = "My order #1234 was for a 'Nova 2000', but it arrived damaged. It was a gift for my sister."

messages = [
    {"role": "user", "content": message_from_user}
]

result = client.add(
    messages,
    user_id=user_id,
    version="v2",
    output_format="v1.1"
)
print(result)

{'results': [{'message': 'Memory processing has been queued for background execution', 'status': 'PENDING', 'event_id': '94b82c27-879e-4c11-afe4-edb6ba12c10f'}]}


### **Получение и поиск памяти**

Можно retrieve (получить) все из памяти о пользователе или искать конкретные контексты, релевантные текущему запросу.

**Вариант А:** Получить все памяти пользователя

In [4]:
filters = {
    "OR": [{"user_id": user_id}]
}

user_memories = client.get_all(filters=filters)
print(user_memories)

{'results': [{'id': 'c616312f-3104-4514-b948-ad3d18db820b', 'memory': 'User is a premium member of the rewards program', 'user_id': 'customer-001', 'metadata': None, 'categories': ['misc'], 'created_at': '2026-02-19T23:56:00.280865-08:00', 'updated_at': '2026-02-19T23:56:00.283910-08:00', 'expiration_date': None, 'structured_attributes': {'day': 20, 'hour': 7, 'year': 2026, 'month': 2, 'minute': 55, 'quarter': 1, 'is_weekend': False, 'day_of_week': 'friday', 'day_of_year': 51, 'week_of_year': 8}}, {'id': 'f0ff3d5e-b9f8-4a40-8ea1-837461820451', 'memory': "User's order #1234 for a Nova 2000 arrived damaged and was intended as a gift for their sister.", 'user_id': 'customer-001', 'metadata': None, 'categories': ['technology', 'family'], 'created_at': '2026-02-19T23:09:35.264587-08:00', 'updated_at': '2026-02-19T23:09:35.266946-08:00', 'expiration_date': None, 'structured_attributes': {'day': 20, 'hour': 7, 'year': 2026, 'month': 2, 'minute': 9, 'quarter': 1, 'is_weekend': False, 'day_of_w

**Вариант Б:** Поиск релевантного контекста (Search)

Когда пользователь задает новый вопрос, найдите соответствующую историю, чтобы агент ответил персонализированно.

In [5]:
new_message = "What's the status on the replacement?"

# Поиск по смыслу
search_memory_response = client.search(
    query=new_message,
    filters=filters,
    version="v2",
    output_format="v1.1"
)

print(search_memory_response)

{'results': [{'id': 'c616312f-3104-4514-b948-ad3d18db820b', 'memory': 'User is a premium member of the rewards program', 'user_id': 'customer-001', 'metadata': None, 'categories': ['misc'], 'created_at': '2026-02-19T23:56:00-08:00', 'updated_at': '2026-02-19T23:56:00-08:00', 'expiration_date': None, 'score': 0.9, 'structured_attributes': {'day': 20, 'hour': 7, 'year': 2026, 'month': 2, 'minute': 55, 'quarter': 1, 'is_weekend': False, 'day_of_week': 'friday', 'day_of_year': 51, 'week_of_year': 8}}, {'id': 'f0ff3d5e-b9f8-4a40-8ea1-837461820451', 'memory': "User's order #1234 for a Nova 2000 arrived damaged and was intended as a gift for their sister.", 'user_id': 'customer-001', 'metadata': None, 'categories': ['technology', 'family'], 'created_at': '2026-02-19T23:09:35-08:00', 'updated_at': '2026-02-19T23:09:35-08:00', 'expiration_date': None, 'score': 0.8181818181818181, 'structured_attributes': {'day': 20, 'hour': 7, 'year': 2026, 'month': 2, 'minute': 9, 'quarter': 1, 'is_weekend': F

Mem0 вернет память о заказе #1234 и товаре Nova 2000 с оценкой релевантности (например, score: 0.9), даже если в новом запросе эти слова не упоминаются напрямую.

### **Продвинутая настройка (Custom Instructions)**

Можно давать указания, какую именно информацию извлекать.

In [6]:
message_from_user_2 = "I'm also a premium member of your rewards program."
custom_instruction = "IDENTIFY AND EXTRACT THE USER'S LOYALTY PROGRAM TIER."

advanced_memory_response = client.add(
    messages=[{"role": "user", "content": message_from_user_2}],
    user_id=user_id,
    custom_instructions=custom_instruction,
    version="v2"
)

# **Mem0 и Qdrant (Архитектура и Self-Hosted)**

**Выбор хранилища**

Когда **Milvus** — лучший выбор:
- Вам нужно хранить много данных (>10M векторов)
- Требуется гибкая фильтрация + векторный поиск
- Важен контроль над данными (on-premise / private cloud)
- Планируете масштабироваться в будущем

Когда можно рассмотреть альтернативы:
- **Pinecone:** если нужен максимально простой старт и вы готовы платить за SaaS
- **Chroma:** для прототипов и небольших проектов (<100K векторов)
- **Qdrant:** если нужен лёгкий self-hosted вариант с хорошим API

**Зачем нужена векторная бд?**

Mem0 хранит воспоминания в виде векторных эмбеддингов. Для поиска похожих воспоминаний (семантический поиск) требуется векторная база данных.
- **В Cloud версии:** Mem0 управляет этим автоматически.
- **В Self-Hosted версии:** Вы разворачиваете Milvus/Qdrant и настраиваете Mem0 на работу с ним.

<br>

|Технология|Применение|
|----------|----------|
|Mem0 Cloud|MVP/Стартап|
|Self-Hosted + Zilliz Cloud.|Enterprise / GDPR|
|Full On-Premise|Self-Hosted + Milvus (Docker)|

###